## 구글 드라이브 연동

In [ ]:
# google drive 연동
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## 드라이브에 업로드한 압축 파일 해제

In [ ]:
# 압축 해제
# %cd /content/drive/MyDrive/숨고/송민채님(언어 모델 비교)/Data
# !unzip -qq "/content/drive/MyDrive/숨고/송민채님(언어 모델 비교)/Data/summarization.zip"

## 데이터 불러오기

In [ ]:
# import library
import json, os, pandas as pd

# path setting
path = '/content/drive/MyDrive/숨고/송민채님(언어 모델 비교)/Data/'

# Data Load
i=1 # i -> 각 종류별 i개씩 테스트
text_data = pd.DataFrame()
for folder_list in os.listdir(path):
    for data_path in os.listdir(path+folder_list+"/20per"):
        with open (path+folder_list+"/20per/"+data_path, "r") as f:
            data = json.load(f)

        temp = pd.DataFrame({'raw':[data['Meta(Refine)']['passage']],'summary':[data['Annotation']['summary3']]})
        text_data = pd.concat([text_data,temp], axis=0)

        if i == 1:
            break;




In [ ]:
text_data.head(10)

,raw,summary
0,여름 동안 시내는 관광객들로 북적인다.\n시내 한가운데 노르웨이 왕궁이 있다.\n1...,시내 한가운데 노르웨이 왕궁이 있다. 1849년에 세워진 왕궁은 화려함보다는 소박함...
0,구자철(31·알 가라파)이 독일 방송에 출연해 한국프로축구 K리그를 홍보했다.\n...,"구자철은 18일과 20일, 자신의 유튜브 채널 ‘슛별친’에 독일 스포츠채널 ‘슈포트..."
0,정부24에서 정부 보조금 정보를 확인하고 신청할 수 있는 (가칭)국가보조금 맞춤형...,정부24에서 정부 보조금 정보를 확인하고 신청할 수 있는 (가칭)국가보조금 맞춤형서...
0,"광산 김씨 예안파 종가의 21대 6백년간에 걸쳐 전해내려오는 고문서들로 1,000여...","이 가운데 교지 등 7종 429점이 보물로 지정되어 있다. 교지, 교서, 교첩, 차..."
0,의원발의법안 비용추계의 주체가 국회예산정책처로 일원화되었다.\n그 동안 의원실에서 ...,그 동안 의원실에서 상당 부분을 추계하던 것을 전문기관이 담당하게 됨에 따라 비용추...
0,"한국교육학술정보원장 임승빈] ""예 알겠습니다.""\n김희정 위원] ""제가 서면질의 드...","김희정 위원] ""제가 서면질의 드린 13번째 서면질의서에 이미 똑같은 질문을 드렸습..."
0,항우와 산동 사람들의 심리상태를 정확히 읽어 낸 한신의 계책을 높이 평가한 유방은...,항우와 산동 사람들의 심리상태를 정확히 읽어 낸 한신의 계책을 높이 평가한 유방은 ...
0,(2) 사원의 배우자 및 피부양자사원의 배우자 및 피부양자가 직접 플랜에 참가할 수...,"사원의 배우자 및 피부양자가 직접 플랜에 참가할 수는 없지만, 사원 자신이 자신의 ..."
0,통신정책관입니다. 과기정통부는 지난 2020년 12월 14일 발생한 구글의 주요 서...,과기정통부는 개정된 전기통신사업법에 따라 구글의 서비스 장애 관련 원인 및 조치계획...
0,‘어떻게 하면 좋을까. 아무래도 서로 만나야겠다. 지금은 오직 서로 만나 직접 이...,경순은 인섭에게서 오는 편지를 한 장도 뜯어보지 않고 그대로 책상 서랍 에 집어넣은...


## 1) KoBART-summarization : 문서 요약 테스크에 최적화

In [ ]:
#!pip install transformers
#!pip install torch

# import library
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

# import model
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

def kobart_summarization(text):
    text = text.replace("\n"," ")

    raw_input_ids = tokenizer.encode(text)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

    summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=int(len(text)*0.2),  eos_token_id=1)
    result = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    return result

text_data['kobart_summary'] = text_data['raw'].apply(lambda x : kobart_summarization(x))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 7.9 MB/s 
     |████████████████████████████████| 120 kB 69.6 MB/s 
     |████████████████████████████████| 6.6 MB 49.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/682k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

## 2) Kogpt2-summarization : 문서 생성 테스크용으로 문서 요약에 적용은 어려움(사례 또한 찾지 못함)

## 3) mt5-summarization

In [ ]:
!pip install sentencepiece

import re
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum", use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum", use_fast=False)

def mt5_summarization(text):
    WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

    input_ids = tokenizer(
        [WHITESPACE_HANDLER(text)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=len(text)
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=int(len(text)*0.2),
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )
    return summary

text_data['mt5_summary'] = text_data['raw'].apply(lambda x : mt5_summarization(x))

## Evaluation

In [ ]:
# 결과 저장
# text_data.to_csv('/content/drive/MyDrive/숨고/송민채님(언어 모델 비교)/result.csv')

#!pip install rouge

from rouge import Rouge
rouge = Rouge()
kobart_rouge = rouge.get_scores(text_data['summary'], text_data['kobart_summary'], avg=True)
mt5_rouge = rouge.get_scores(text_data['summary'], text_data['mt5_summary'], avg=True)

total_result = pd.DataFrame({'rouge-1_recall':[kobart_rouge['rouge-1']['r'], mt5_rouge['rouge-1']['r']],
                             'rouge-1_precision':[kobart_rouge['rouge-1']['p'], mt5_rouge['rouge-1']['p']],
                             'rouge-1_f1score':[kobart_rouge['rouge-1']['f'], mt5_rouge['rouge-1']['f']],
                             'rouge-2_recall':[kobart_rouge['rouge-2']['r'], mt5_rouge['rouge-2']['r']],
                             'rouge-2_precision':[kobart_rouge['rouge-2']['p'], mt5_rouge['rouge-2']['p']],
                             'rouge-2_f1score':[kobart_rouge['rouge-2']['f'], mt5_rouge['rouge-2']['f']],
                             'rouge-l_recall':[kobart_rouge['rouge-l']['r'], mt5_rouge['rouge-l']['r']],
                             'rouge-l_precision':[kobart_rouge['rouge-l']['p'], mt5_rouge['rouge-l']['p']],
                             'rouge-l_f1score':[kobart_rouge['rouge-l']['f'], mt5_rouge['rouge-l']['f']]}, index=['KoBART', 'MT5']).round(2)
total_result

,rouge-1_recall,rouge-1_precision,rouge-1_f1score,rouge-2_recall,rouge-2_precision,rouge-2_f1score,rouge-l_recall,rouge-l_precision,rouge-l_f1score
KoBART,0.63,0.43,0.49,0.55,0.37,0.43,0.62,0.43,0.49
MT5,0.27,0.05,0.09,0.13,0.02,0.04,0.27,0.05,0.09
